In [2]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

In [3]:
symbols = ['AAPL','MSFT','META','NVDA']
end_date = datetime.now()
start_date = datetime(2020,1,1)
data = vbt.YFData.download(
    symbols = symbols,    
    interval = '1d',
    start = start_date, 
    end = end_date,
).get('Close')
data

/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Setting missing data points to NaN.

/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Setting missing data points to NaN.



symbol,AAPL,MSFT,META,NVDA
Date,,,,
2019-12-31 05:00:00+00:00,70.920380,150.013489,203.992462,5.856666
2020-01-02 05:00:00+00:00,72.538536,152.791138,208.494659,5.971411
2020-01-03 05:00:00+00:00,71.833275,150.888596,207.391495,5.875832
2020-01-06 05:00:00+00:00,72.405685,151.278610,211.297409,5.900472
2020-01-07 05:00:00+00:00,72.065147,149.899307,211.754593,5.971908
...,...,...,...,...
2025-10-03 04:00:00+00:00,258.019989,517.349976,710.559998,187.619995
2025-10-06 04:00:00+00:00,256.690002,528.570007,715.659973,185.539993
2025-10-07 04:00:00+00:00,256.480011,523.979980,713.080017,185.039993


In [ ]:
def _sma(data:pd.DataFrame, window: int):
    sma = data.rolling(window = window).mean()
    return sma

def apply_sma(data:pd.DataFrame, fast:int, slow:int):
    # print(data)
    fast_sma = _sma(data,window = fast)
    slow_sma = _sma(data,window = slow)
    trend = pd.DataFrame(0, index=data.index, columns=data.columns)
    trend[fast_sma < slow_sma] = -1
    trend [fast_sma > slow_sma] = 1 
    trend = trend.diff() #2 for buy, -2 for sell
    
    return fast_sma, slow_sma, trend

sma_indicator = vbt.IndicatorFactory(
    class_name = 'CustomSMA',
    short_name = 'SMA',
    input_names = ['data'],
    param_names = ['fast','slow'],
    output_names = ['fast_sma','slow_sma','trend' ]
).from_apply_func(apply_sma)

fast_windows = [10, 20, 30]
slow_windows = [50, 100, 150]

results = sma_indicator.run(data, 
                            fast = fast_windows, 
                            slow = slow_windows, 
                            keep_pd = True)
entries = results.trend == 2
exits = results.trend == -2
pf = vbt.Portfolio.from_signals(data, 
                                entries, 
                                exits,
                                fees=0.001,  
                                size=1_000,  # trade size per asset
                                init_cash=100_000
                                )
returns = pf.total_return()
returns




/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/tmp/ipykernel_3964/920171947.py:39: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x77f873a0ba60>. Pass column to select a single column/group.



KeyError: 'Sharpe Ratio'